In [1]:
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from IPython.display import Image, display
import nltk
from nltk.classify import apply_features
from nltk import word_tokenize
from collections import Counter
import random
import time
import re
from datetime import datetime
import pylast
import spotipy
import spotipy.util as util
import urllib3
fs = 16 # fontsize

## Load and format the lyrics JSON object

In [2]:
artists_rap = json.load(open('lyrics/rap_lyrics.json'))

In [3]:
artists_rock = json.load(open('lyrics/rock_lyrics.json'))

In [4]:
artists_country = json.load(open('lyrics/country_lyrics.json'))

In [5]:
artists_raw = artists_rap + artists_rock + artists_country

In [6]:
# Determine
def getArtistImage(songs):    
    urls = [None]
    for song in songs:
        if song['raw']['album'] is not None:
            try:                
                urls.append(song['raw']['album']['artist']['image_url'])                
            except:
                pass
    
    return Counter(urls).most_common()[0][0] # Most common image

# Use a generator to exclude artists with too few songs
def filterBySongCount(artists, min_count=10, max_count=10000):
    for artist in artists:        
        num_songs = len(artist['songs'])
        if (num_songs >= min_count) and (num_songs < max_count): yield artist

min_songs, max_songs = 10, 2500
artists = list(filterBySongCount(artists_raw, min_songs, max_songs))

# Artist images
for artist in artists:
    artist['image_url'] = getArtistImage(artist['songs'])
    
    # Kludge
    if artist['artist'] == 'Black Thought':
        artist['image_url'] = 'https://images.genius.com/23de511f4ac50d900128ea363b7d81b3.450x320x1.jpg'
    if artist['artist'] == 'André 3000':
        artist['image_url'] = 'http://images.genius.com/64b15c9489c65f5bf8f6577334347404.434x434x1.jpg'            
        
# Gender
labels_gender = np.array([a['gender'] for a in artists])
mask_female = np.array([g=='female' for g in labels_gender])
mask_male   = np.array([g=='male'   for g in labels_gender])
print("You have lyrics from {} artists, with a minimum of {} songs per artist.\nFemale: {}, male: {}"
      .format(len(artists), min_songs, (labels_gender=='female').sum(), (labels_gender=='male').sum()))

# Add all song lyrics to a single list
all_songs, genres = [], []
for artist, gender in zip(artists, labels_gender):
    for song in artist['songs']:        
        song['gender'] = gender
        if len(song['lyrics'].split(' ')) > 10:
            all_songs.append(song)
            if song['genre'] not in genres:
                genres.append(song['genre'])
              
song_count = len(all_songs)
print("Database contains {} songs.".format(song_count))

# Store all lyrics in a single string
all_lyrics = " ".join([song['lyrics'] for song in all_songs])

You have lyrics from 303 artists, with a minimum of 10 songs per artist.
Female: 79, male: 222
Database contains 56907 songs.


## Load the last.fm API credentials

In [7]:
with open('secrets_lastfm.txt','r') as f:
    lines = [line.split(': ') for line in f.readlines()]
    API_KEY = lines[1][1].strip()
    API_SECRET = lines[2][1].strip()        
    username = lines[3][1].strip()    
    password = lines[4][1].strip()

password_hash = pylast.md5(password)
lastfm = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET,
                              username=username, password_hash=password_hash)

## Load the Spotify API credentials

In [8]:
with open('secrets_spotify.txt','r') as f:
    lines = [line.split(': ') for line in f.readlines()]
    spotify_client_id = lines[0][1].strip()
    spotify_client_secret = lines[1][1].strip()    
    spotify_username = lines[2][1].strip()
    
scope = 'user-library-read'
spotify_token = util.prompt_for_user_token(spotify_username, scope, client_id=spotify_client_id,
                           client_secret=spotify_client_secret,
                           redirect_uri='http://localhost/')

# Create the spotipy object
sp = spotipy.Spotify(spotify_token)

In [9]:
import spotipy.oauth2 as oauth

In [10]:
spo = oauth.SpotifyOAuth(client_id=spotify_client_id,
                   client_secret=spotify_client_secret,
                   redirect_uri='http://localhost/',
                   scope=scope,
                   cache_path=".cache-{}".format(username))

In [11]:
def get_spotify_token():
    token_info = spo.get_cached_token()
    if token_info:
        access_token = token_info['access_token']
        return access_token
    else:
        auth = spo.get_authorize_url()
        print(auth)
        auth_url = input('Click the link above and copy and paste the url here: ')
        _re_auth = re.findall(_auth_finder, auth_url)
        access_token = self.spo.get_access_token(_re_auth[0])
        return access_token    

In [12]:
def refresh_spotify_token():
    cached_token = spo.get_cached_token()
    refreshed_token = cached_token['refresh_token']
    new_token = spo.refresh_access_token(refreshed_token)        
    return new_token

## Get duration and date information for all songs

In [13]:
def song_is_lyrics(song_title):
    """Returns False if result from Genius is not actually song lyrics"""
    regex = re.compile(
        r"(booklet)|(tracklist)|(track list)|(album art(work)?)|(liner notes)|(booklet)|(credits)|(remix)|(interview)|(skit)", re.IGNORECASE)
    return not regex.search(song_title)

In [478]:
artist_name

'The Beatles'

In [476]:
j

490

In [477]:
i

185

In [ ]:
# I should have just checked for Media first...
artists[0]['songs'][0]['raw']['media'][1]

In [28]:
spotify_artists = json.load(open('./spotify_data_v235.json'))

In [30]:
artists[236]['artist']

'Linda Ronstadt'

In [34]:
skip_num = 236
start = time.time()
time_timeout = start
for i,artist in enumerate(artists):
    success_count, fail_count = 0,0
    artist_name = artist['artist']
    if artist_name not in [artist['artist'] for artist in artists[:(skip_num+1)]]:
        spotify_artists.append({'artist': artist_name, 'songs': []})
    print("{line}\nArtist: {name}, ({n}/{T})".format(line="-"*60, name=artist_name, n=i,T=len(artists)))
    for j,song in enumerate(artist['songs']):
        if (artist_name not in [artist['artist'] for artist in artists[:skip_num+1]]) or (artist_name=='Linda Ronstadt' and j>0):
            duration,year = None, None
            song_title = song['title']
            per = 100*success_count/(success_count+fail_count) if (success_count+fail_count)>0 else 0
            if j%10 == 0:
                print("Song: {title} || ({j}/{tot}), (s={s},f={f},p={p:.2f})".format(title=song_title, j=j, tot=len(artist['songs']),
                     s=success_count, f=fail_count, p=per))

            if song_is_lyrics(song_title): # reject non-songs            
                album = song['album'] if song['album'] is not None else ""            
                try: # Try Spotify first
                    q = "{song_title} {artist_name} {album}".format(song_title=song_title, artist_name=artist_name, album=album)
                    search_result = sp.search(q, limit=1, type='track')
                    if search_result['tracks']['total'] != 0:
                        try: 
                            track = search_result['tracks']['items'][0]                
                            duration = track['duration_ms']/1e3 # convert to seconds                
                            if 'release_date' in track['album']:
                                year = track['album']['release_date']
                            spotify_artists[i]['songs'].append({'lyrics': song['lyrics'],
                                                                'title': song_title,
                                                                'album': album,
                                                                'spotify': track})
                            artists[i]['songs'][j]['spotify'] = track
                            
                        except Exception as e:
                            print("ERROR (sp): {}".format(e))
                    else: # Search for song on last.fm if song not found on Spotify
                        try:
                            track = lastfm.get_track(artist_name, song['title'])
                            duration = track.get_duration()/1e3 # convert to seconds
                            duration = None if duration==0 else duration
                        except Exception as e:
                            if e.details != "Track not found":                        
                                print("ERROR (fm): {}".format(e))
                except Exception as e:
                    print("ERROR (SP): {}".format(e))
                    print("Time since last timeout: {:.2f} minutes".format((time.time()-time_timeout)/60))
                    time_timeout = time.time()
                    sp = spotipy.Spotify(auth=refresh_spotify_token()['access_token'])

                # Assign the duration and year values to the artist's song
                artists[i]['songs'][j]['duration'] = duration
                if year is not None:
                    artists[i]['songs'][j]['year'] = year

                if duration is None:
                    fail_count += 1
                else:
                    success_count += 1
                    print("{title}: ({dur}, {year})".format(title=song_title,
                                                          dur=artists[i]['songs'][j]['duration'],
                                                          year=artists[i]['songs'][j]['year']))

                # Wait before calling API again
                time.sleep(2)
    print("Total songs added for {}: {}, Final accuracy: {:.2f}".format(artist_name, success_count, per))
    
    # Save the spotify artists object
    if i%5 == 0 and i >= skip_num:
        filename = 'spotify_data_v{}'.format(i)
        with open(filename + '.json', 'w') as outfile:
            json.dump(spotify_artists, outfile)
            
end = time.time()
print("Time elapsed: {} hours".format((end-start)/60/60))

------------------------------------------------------------
Artist: 2Pac, (0/303)
Total songs added for 2Pac: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Eminem, (1/303)
Total songs added for Eminem: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: The Notorious B.I.G., (2/303)
Total songs added for The Notorious B.I.G.: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Nas, (3/303)
Total songs added for Nas: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Ice Cube, (4/303)
Total songs added for Ice Cube: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: JAY-Z, (5/303)
Total songs added for JAY-Z: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Snoop Dogg, (6/303)
Total songs added for Snoop Dogg: 0, Final ac

Total songs added for E-40: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Pusha T, (71/303)
Total songs added for Pusha T: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Coolio, (72/303)
Total songs added for Coolio: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: De La Soul, (73/303)
Total songs added for De La Soul: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Proof, (74/303)
Total songs added for Proof: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Bad Meets Evil, (75/303)
Total songs added for Bad Meets Evil: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Guru, (76/303)
Total songs added for Guru: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Will 

Total songs added for Blue Öyster Cult: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Deep Purple, (139/303)
Total songs added for Deep Purple: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Green Day, (140/303)
Total songs added for Green Day: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Neil Young, (141/303)
Total songs added for Neil Young: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Tom Petty and the Heartbreakers, (142/303)
Total songs added for Tom Petty and the Heartbreakers: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Foo Fighters, (143/303)
Total songs added for Foo Fighters: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Grateful Dead, (144/303)
Total songs added for Grateful Dead: 0,

Total songs added for David Bowie: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Chuck Berry, (210/303)
Total songs added for Chuck Berry: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Coldplay, (211/303)
Total songs added for Coldplay: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: The Doors, (212/303)
Total songs added for The Doors: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Jethro Tull, (213/303)
Total songs added for Jethro Tull: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Metallica, (214/303)
Total songs added for Metallica: 0, Final accuracy: 100.00
------------------------------------------------------------
Artist: Queen, (215/303)
Total songs added for Queen: 0, Final accuracy: 100.00
-----------------------------------------------

Hobo: (198.933, 1987-03-03)
Honky Tonk Blues: (146.0, None)
Song: How Do I Make You || (100/296), (s=98,f=1,p=98.99)
How Do I Make You: (144.773, 1980)
Hummin' to Myself: (265.08, 2004-01-01)
Hurt So Bad: (196.0, 1980)
I Ain't Always Been Faithful: (171.6, 1972)
I Believe in You: (170.706, 1973)
I Can Almost See It: (230.84, 1973)
I Can't Get Over You [With Ann Savoy]: (187.0, None)
I Can't Help It (If I'm Still in Love With You): (165.933, 1974)
I Can't Let Go: (159.6, 1980)
Icy Blue Heart: (297.733, 1998-06-12)
Song: I Don't Stand a Ghost Of A Chance || (110/296), (s=108,f=1,p=99.08)
I Don't Stand a Ghost Of A Chance: (249.2, 1986)
I Fall in Love Too Easily: (216.253, 2004-01-01)
I Fall to Pieces: (191.04, 1972)
If He's Ever Near: (194.466, 1976)
If I Should Fall Behind: (246.266, 1998-06-12)
I Get Along Without You Very Well: (257.56, 1986-09-15)
I Go to Pieces: (227.04, 1998-06-12)
I Just Don't Know What to Do With Myself: (191.173, 1993-11-09)
I Keep It Hid: (215.0, None)
I Knew Y

...

Raining On Sunday: (285.6, 2002-10-04)
Raise Em Up: (184.946, 2013-01-01)
Raise The Barn: (311.186, 2006-11-03)
Red Camaro: (239.28, 2013-01-01)
Right On Back To You: (288.813, 2011-02-22)
Rollercoaster: (171.0, 1999-10-19)
Romeo's Tune: (227.2, 2007-11-20)
Same Heart*: (216.261, 2018-04-27)
Shame: (183.893, 2013-01-01)
Song: She's Gotta Be || (90/142), (s=89,f=1,p=98.89)
She's Gotta Be: (292.093, 2004-09-21)
She's My 11: (197.48, 2013-01-01)
Shine: (317.12, 2006-11-03)
Shut Out The Lights: (258.173, 2011-02-22)
Somebody Like You: (323.04, 2002-10-04)
Some Days You Gotta Dance: (149.693, 1997)
Somewhere In My Car: (236.906, 2013-01-01)
Song For Dad: (236.453, 2002-10-04)
Standing Right In Front Of You: (241.426, 2009-03-31)
Stuck In The Middle: (229.96, 1997)
Song: Stupid Boy || (100/142), (s=99,f=1,p=99.00)
Stupid Boy: (376.386, 2006-11-03)
Sun Don't Let Me Down: (203.133, 2016-05-06)
Sweet Thing: (228.186, 2009-03-31)
Tangled Up In Love: (230.96, 1997)
Thank You: (314.76, 2009-03-31)

Cold Shoulder: (236.0, 1991-09-02)
Cowboy Bill: (276.04, 2000)
Cowboy Cadillac: (170.0, None)
Cowboys and Angels: (196.0, 1995-11-21)
Cowboys and Friends: (195.0, 2016-11-25)
Song: Cowgirl's Saddle || (30/187), (s=27,f=3,p=90.00)
Cowgirl's Saddle: (195.0, None)
Digging For Gold: (308.0, None)
Dixie Chicken: (264.0, 1992-09-14)
Don't Cross The River: (263.0, None)
Do What You Gotta Do: (177.0, None)
Driftin' Away: (296.0, None)
Every Now And Then: (255.0, 1992-09-14)
Everytime That it Rains: (247.0, 1989-04-12)
Face To Face: (264.0, 1992-09-14)
Song: Fish || (40/187), (s=36,f=4,p=90.00)
Fish: (276.0, 2014-11-11)
Fishin' In The Dark: (222.0, None)
Fit For A King: (238.0, None)
For a Minute There: (316.0, None)
Friends in Low Places: (304.323, 2012-04-18)
God Rest Ye Merry Gentlemen: (155.0, None)
Good Ride Cowboy: (206.0, None)
Go Tell It On The Mountain: (210.0, None)
Hard Luck Woman: (189.445, 2015-10-12)
Have Yourself A Merry Little Christmas: (246.0, None)
Song: He Really Loves You |

Aw Naw: (188.706, 2013-09-16)
Beer or Gasoline: (208.906, 2006-09-30)
Blacked Out: (189.88, 2017-10-20)
Burn: (183.92, 2006-09-30)
Callin' My Name: (191.12, 2015-11-13)
Center Of My World: (213.986, 2006-09-30)
Drinkin' Me Lonely: (199.36, 2006-09-30)
Flashlight: (202.306, 2011-07-11)
Song: Flowers || (10/69), (s=10,f=0,p=100.00)
Flowers: (180.493, 2006-09-30)
Forgiveness: (211.0, None)
Gettin' You Home: (212.133, 2009-08-27)
Gettin' You Home (The Black Dress Song): (212.0, None)
Goodbye: (224.613, 2013-09-16)
Hangin' On: (185.44, 2017-10-20)
Heartbeat: (175.386, 2015-11-13)
Hold You to It: (197.106, 2013-09-16)
Holiday: (176.4, 2017-10-20)
I Can Take It From There: (158.293, 2011-07-11)
Song: I Know A Guy || (20/69), (s=20,f=0,p=100.00)
I Know A Guy: (243.32, 2015-11-13)
I'm Comin' Over: (197.28, 2015-11-13)
I'm Headed Your Way, Jose: (209.106, 2006-09-30)
It Takes a Man: (212.48, 2009-08-27)
Lay It On Me: (210.773, 2006-09-30)
Leave Me Wanting More: (180.96, 2017-10-20)
Lighters in t

I Wanna Be A Hillbilly: (186.253, 2005-01-01)
Jonesin': (211.959, 2015-06-02)
Keep Your Hands To Yourself: (207.0, None)
Song: Let Me Down Easy || (30/67), (s=30,f=0,p=100.00)
Let Me Down Easy: (228.493, 2010-01-01)
Life And Love And The Meaning Of: (219.68, 2008-01-01)
Like My Dog: (166.56, 2010-01-01)
Lil' Ol' Lonesome Dixie Town: (201.666, 2010-01-01)
Little Bit Lonely: (225.8, 2005-01-01)
Love Done Gone: (210.746, 2010-01-01)
Lucille: (248.48, 2005-01-01)
Must Be Doin' Somethin' Right: (269.586, 2005-01-01)
Next Time: (195.906, 2003-01-01)
No One Has Eyes Like You: (178.026, 2008-01-01)
Song: Nowhere Town || (40/67), (s=40,f=0,p=100.00)
Nowhere Town: (222.759, 2015-06-02)
Off My Rocker: (189.133, 2003-01-01)
One Way Ticket: (175.0, 2013-09-17)
People Are Crazy: (232.0, 2008-10-14)
Perfect Day: (174.52, 2010-01-01)
Pretty Good At Drinkin' Beer: (181.0, 2010-09-21)
She Knows What To Do With A Saturday Night: (218.44, 2005-01-01)
She's Got A Way With Me: (285.333, 2005-01-01)
Soundtra

To The Sky: (205.586, 2014-09-09)
Song: Unwind It || (30/42), (s=30,f=0,p=100.00)
Unwind It: (197.733, 2012-08-21)
Waiting: (218.933, 2012-08-21)
What You Wanna Hear: (211.68, 2014-09-09)
Where It's At: (204.186, 2014-09-09)
Why Not Tonight: (185.573, 2017-09-08)
Why We Call Each Other: (200.36, 2017-09-08)
Wild In Your Smile: (180.146, 2012-08-21)
World To Me: (209.733, 2014-09-09)
Yeah Yeah Yeah: (197.933, 2012-08-21)
Song: Your Daddy's Boots || (40/42), (s=39,f=1,p=97.50)
Your Daddy's Boots: (230.52, 2014-09-09)
Your Plan: (209.653, 2012-08-21)
Total songs added for Dustin Lynch: 41, Final accuracy: 97.56
------------------------------------------------------------
Artist: Dierks Bentley, (301/303)
Song: 5-1-5-0 || (0/120), (s=0,f=0,p=0.00)
5-1-5-0: (182.893, 2012-02-03)
A house of gold: (266.0, None)
All the Way to Me: (218.2, 2016-06-10)
Always Be Me: (247.0, None)
Am I The Only One: (191.253, 2012-02-03)
Back Porch: (191.44, 2014-01-01)
Bad Angel: (244.16, 2010-06-04)
Band Of Bro

In [35]:
filename = 'spotify_data_b7'
with open(filename + '.json', 'w') as outfile:
    json.dump(spotify_artists, outfile)
